In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/stock_price_4_fun/notebook


In [2]:
import shutil
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)
from src.trading_record import *

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service


In [50]:

class TradeScraper:
    def __init__(self, username, password):
        # How to run selenium on linux
        # https://cloudbytes.dev/snippets/run-selenium-and-chrome-on-wsl2#:~:text=With%20Selenium%20libraries%2C%20Python%20can,using%20Python%20and%20Selenium%20webdriver.
        self.username = username
        self.password = password
        chrome_options = Options()
        chrome_options.add_argument("--headless") # Ensure GUI is off
        chrome_options.add_argument("--no-sandbox")
        homedir = os.path.expanduser("~")
        webdriver_service = Service(f"{homedir}/chromedriver/stable/chromedriver-linux64/chromedriver")

        # Choose Chrome Browser
        self.driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)

        # self.driver = webdriver.Chrome(options=options)
        # self.driver = webdriver.Chrome()

    def signin(self):
        # Step 1: Navigate to URL
        self.driver.get("https://accounts.fpts.com.vn/Login?href=eztrade")

        # Step 2: Find and fill in the username and password fields
        username_input = self.driver.find_element(By.ID, 'txtAccountNo')
        password_input = self.driver.find_element(By.ID, "txtPassword")

        username_input.send_keys(self.username)
        password_input.send_keys(self.password)

        # Step 3: Submit the form
        password_input.send_keys(Keys.RETURN)

    def scrape_buy_sell_log(self, start_date = "27/10/2023", end_date = "27/11/2023"):
        # Step 1: Navigate to TradeLog URL
        self.signin()
        self.driver.get("https://eztrade.fpts.com.vn/report/TradeLog")

        # Step 2: Find and fill in the start and end date fields
        start_date_input = self.driver.find_element(By.ID, 'txtDateFrom')
        end_date_input = self.driver.find_element(By.ID, 'txtDateTo')

        start_date_input.send_keys(start_date)
        # end_date_input.send_keys(end_date)


        # Step 3: Find the button or link to download the report
        download_button = self.driver.find_element(By.ID, "imgExcel_CA")

        # Step 4: Click the download button
        download_button.click()
        download_file = 'LichSuKhopLenh_058C' + self.username + '.xls'
        source_folder = os.path.expanduser("./")  # Assuming the default Downloads folder
        destination_folder = "data"  # Specify your destination folder
        self.moveDownloadedFile(source_folder, destination_folder, download_file)
        self.rename_xls_html(file_path=f'data/{download_file}')

    def moveDownloadedFile(self, source_folder, destination_folder, file_name):
        source_path = os.path.join(source_folder, file_name)
        shutil.move(source_path, destination_folder)

    def rename_xls_html(self, file_path ="data/LichSuKhopLenh_058C647873.xls"):
        file_name, file_extension = os.path.splitext(file_path)
        html_file_name = file_name + ".html"
        shutil.move(file_path, html_file_name)
        df = pd.read_html(html_file_name)[0]
        df.to_csv(file_name + '.csv', index=False)
        # os.remove(html_file_name)
    
    def closeBrowser(self):
        # Close the browser window
        self.driver.quit()

In [51]:
TRADE_USER= os.getenv('TRADE_USER')
TRADE_PASS= os.getenv('TRADE_PASS')
# Create an instance of TradeScraper
scraper = TradeScraper(TRADE_USER, TRADE_PASS)
scraper.scrape_buy_sell_log(start_date="20/11/2023", end_date="27/11/2023")
scraper.closeBrowser()

In [52]:
buysell_analyzer = BuySellAnalyzer(buy_sell_df_path='data/LichSuKhopLenh_058C647873.csv')

In [53]:
buysell_analyzer.df.tail()

,time,action,ticker,vol,price,value,tax_from_transfer,tax_from_capital,platform_fee,total_fee
83,2023-11-24,sell,HHV,400.0,14200.0,5680000.0,5680.0,0.0,4999.0,10679.0
85,2023-11-24,buy,GEX,300.0,21400.0,6420000.0,0.0,0.0,5649.0,5649.0
87,2023-11-27,sell,HHV,200.0,15000.0,3000000.0,3000.0,0.0,2640.0,5640.0
89,2023-11-28,buy,PDR,200.0,26000.0,5200000.0,0.0,0.0,4576.0,4576.0
90,2023-11-28,buy,VIX,400.0,15800.0,6320000.0,0.0,0.0,5561.0,5561.0


In [54]:
symbol ='VIX'

In [55]:
buysell_analyzer.plot_buy_sell_of_stock(symbol)